In [1]:
import tensorflow as tf
import numpy as np
import pylab as pl
import pandas as pd

%matplotlib inline

In [2]:
from sklearn.datasets import california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

In [3]:
california=california_housing.fetch_california_housing()

In [69]:
x=scale(california['data'])
y=california['target']

df=pd.DataFrame(x,columns=california['feature_names'])

In [70]:
# Splitting into train and test sets

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3)

# Random Forest regressor

* Benchmark system
* RF generally gives great performance for problems like this

In [71]:
xtrain.shape

(14448, 8)

In [72]:
rf=RandomForestRegressor(n_estimators=100).fit(xtrain,ytrain)

In [73]:
# MSE Error
np.mean((rf.predict(xtest)-ytest)**2)

0.26596963096238341

# "Deep" Neural Network

* Two hidden layers
* Investigate the effect of different params

In [20]:
nhidden1=250
nhidden2=200
n_iter=1000
activation=tf.nn.relu
#activation=tf.nn.sigmoid

In [141]:
def CreateDN(x,dropKeep,numUnits):
   
    # Setting things up
    nIn=int(x.shape[1])
    y=x
    l2=0
    
    for layer in range(len(numUnits)):
        
        w=tf.get_variable("w"+str(layer),initializer=tf.glorot_normal_initializer(),shape=(nIn,numUnits[layer]))
        b=tf.get_variable('b'+str(layer),initializer=tf.zeros_initializer(),shape=(numUnits[layer]))
        a=tf.add(tf.matmul(y,w),b,name="a"+str(layer))
        if layer==len(numUnits)-1:
            y=a
        else:
            y=tf.nn.relu(a)
            y=tf.nn.dropout(y,keep_prob=dropKeep)
    
        l2+=tf.reduce_sum(w**2)
    
        # Prepping for next layer, if present
        nIn=numUnits[layer]
        
    return y,l2

In [151]:
tf.reset_default_graph()

n_iter=1500

nx=tf.placeholder(tf.float32,(None,8))
dk=tf.placeholder(tf.float32,())
target=tf.placeholder(tf.float32,(None,1))

ny,l2=CreateDN(nx,dk,[256,160,64,1])
testloss=tf.losses.mean_squared_error(ny,target)
loss=testloss+l2*.0
opt=tf.train.AdamOptimizer(learning_rate=.01).minimize(loss)

In [152]:
try: sess.close()
except: pass

sess=tf.Session()
    
writer = tf.summary.FileWriter("/home/wlwoon/tmp/day14log", sess.graph)
sess.run(tf.global_variables_initializer())
    
for count in range(n_iter):
        
    _,tloss=sess.run([opt,loss],feed_dict={dk:.6,nx:xtrain,target:ytrain.reshape((-1,1))})
      
    if count % 50 ==0:
        test_loss=sess.run(testloss,feed_dict={dk:1.0,nx:xtest,target:ytest.reshape((-1,1))})
        print("iter #"+str(count)+", training loss is "+str(tloss)+", test loss is "+str(test_loss))

writer.flush()

iter #0, training loss is 5.3795, test loss is 2.70036
iter #50, training loss is 0.517459, test loss is 0.395986
iter #100, training loss is 0.446613, test loss is 0.338995
iter #150, training loss is 0.409337, test loss is 0.31889
iter #200, training loss is 0.397643, test loss is 0.310944
iter #250, training loss is 0.369275, test loss is 0.311057
iter #300, training loss is 0.362997, test loss is 0.291713
iter #350, training loss is 0.346487, test loss is 0.292763
iter #400, training loss is 0.323148, test loss is 0.278784
iter #450, training loss is 0.323492, test loss is 0.294626
iter #500, training loss is 0.312681, test loss is 0.28051
iter #550, training loss is 0.309541, test loss is 0.284968
iter #600, training loss is 0.298976, test loss is 0.282553
iter #650, training loss is 0.289716, test loss is 0.27268
iter #700, training loss is 0.293659, test loss is 0.281345
iter #750, training loss is 0.287884, test loss is 0.279432
iter #800, training loss is 0.283981, test loss i

In [153]:
sess.run(ny,feed_dict={dk:1,nx:xtest[:5,:]})

array([[ 2.38776541],
       [ 3.07113528],
       [ 2.02339411],
       [ 2.4841907 ],
       [ 1.08343816]], dtype=float32)

In [130]:
np.mean(ytest**2)
np.std(xtest,axis=0)

array([ 1.02065714,  0.99876618,  0.84249565,  0.74652804,  1.00117305,
        1.54552267,  1.00237812,  0.99950791])